In [1]:
from drain.model import Drain,Node,LogCluster
from drain.optimizer import Optimizer
from drain.plotter import createPlot
from drain.partition import Partition
from utils.data_utils import *
import unsupervised.model as unsupervised_model
from torch import nn,optim
from torch.utils.data import  DataLoader
from utils.bert_utils import *
import supervised.model as supervised_model
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 2
batch_size = 2048
input_size = 768
window_size = 10
num_layers = 2
hidden_size = 64

In [3]:
logPath = 'D:\\毕业设计\\loghub\\HDFS_1\\HDFS.log'
parsed_result = '.\\data\\parser\\log_item_to_label.csv'
cluster_result = '.\\data\\parser\\'
# partition
partition_output = '.\\data\\lstm\\dataset_official\\'
instance_file = 'instance.csv'
label_file = '.\\data\\partition\\anormaly_label.csv'
normal_output = 'normal.csv'
abnormal_output = 'abnormal.csv'

#lstm dataset
lstm_dataset = '.\\data\\lstm\\dataset_official\\'
unsupervised_train = 'train.csv'
unsupervised_test_normal = 'normal.csv'
unsupervised_test_abnormal = 'abnormal.csv'

predicted_input = 'unsupervised_input.csv'
predicted_normal_output = 'unsupervised_normal_output.csv'
predicted_abnormal_output = 'unsupervised_abnormal_output.csv'
supervised_test = 'supervised_test.csv'

In [28]:
len(test_dataset)

7111

In [29]:
train_dataset, test_dataset=generate_bert_data(lstm_dataset,predicted_normal_output,predicted_abnormal_output,"./data/lstm/bert_official_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 1895/1895 [00:51<00:00, 36.79it/s]


Number of sessions(.\data\lstm\dataset_official\): 4778
Number of normal sessions: 2883
Number of abnormal sessions: 1895


In [42]:
batch_size = 50
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validate_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [43]:
train_dataloader_ = DataLoader(train_dataset, batch_size=20000, shuffle=True)
validate_dataloader_ = DataLoader(test_dataset, batch_size=20000)

In [44]:
model = supervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [45]:
model_sl_name = 'supervised_lstm_dev_v1'
writer = SummaryWriter(log_dir='log/' + model_sl_name)

In [52]:
import numpy as np
def accuracy(y_pred, y_true):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    pred = np.argmax(y_pred.cpu().numpy(),1)
    for i,val in enumerate(pred):
        if val == 0:
            if y_true[i]==0:
                TN +=1
            else:
                FN +=1
        else:
            if y_true[i]==1:
                TP+=1
            else:
                FP+=1
    return TP,TN,FP,FN

def predict_result(model,data):
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        for step, (seq, label) in enumerate(data):
            seq = seq.clone().detach().view(-1, seq.shape[1], input_size).to(device)
            test_output = model(seq.to(device))
            if step == 0:
                output = test_output
                labels = label
            else:
                labels = torch.cat([labels, label], 0)
                output = torch.cat([output, test_output], 0)
            epoch_loss += criterion(test_output, label.to(device)).data
        TP,TN,FP,FN = accuracy(output, labels)
        A = 100*(TP+TN)/(FP+FN+TP+TN)
        if TP+FP==0:
            P = 100
        else:
            P = round((100*TP/(TP+FP)),3)
        
        
        if TP+FN==0:
            R=100
        else:
            R = round(100*TP/(TP+FN),3)
        F1 = 2 * P * R / (P + R)
        print('TN:{} FN:{} TP:{} FP:{}'.format(TN,FN,TP,FP))
        print('accuracy:{} recall:{}'.format(round(A,3),R))
        return A,R,P,F1

In [53]:
current_epoch = 0
num_epochs = 300
torch.cuda.empty_cache()

In [54]:
total_step = len(train_dataloader)
# model.train()
start_time = time.time()
for epoch in range(current_epoch,num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    
    print("训练集")
    A,R,P,F1 = predict_result(model,train_dataloader_)
    writer.add_scalar('训练集正确率', A, epoch)
    writer.add_scalar('训练集召回率', R, epoch)
    writer.add_scalar('训练集精度', P, epoch)
    writer.add_scalar('训练集F1值', F1, epoch)
    print("验证集")
    A,R,P,F1 = predict_result(model,validate_dataloader_)
    writer.add_scalar('验证集正确率', A, epoch)
    writer.add_scalar('验证集召回率', R, epoch)
    writer.add_scalar('验证集精度', P, epoch)
    writer.add_scalar('验证集F1值', F1, epoch)
    print("测试集")
    A,R,P,F1 = predict_result(model,test_dataloader)
    writer.add_scalar('测试集集正确率', A, epoch)
    writer.add_scalar('测试集集召回率', R, epoch)
    writer.add_scalar('测试集集精度', P, epoch)
    writer.add_scalar('测试集集F1值', F1, epoch)
    model.train()
    for step, (seq, label) in enumerate(train_dataloader):
        # Forward pass
        seq = seq.clone().detach().view(-1, seq.shape[1], input_size).to(device)
        output = model(seq)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        # writer.add_graph(model, seq)
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
    writer.add_scalar('第二阶段模型损失', train_loss / total_step, epoch + 1)

    
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))

训练集
TN:1740 FN:1126 TP:0 FP:0
accuracy:60.712 recall:0.0
验证集
TN:1143 FN:769 TP:0 FP:0
accuracy:59.78 recall:0.0
测试集
TN:4994 FN:2117 TP:0 FP:0
accuracy:70.229 recall:0.0
Epoch [1/300], train_loss: 0.6427
训练集
TN:1642 FN:925 TP:201 FP:98
accuracy:64.306 recall:17.851
验证集
TN:1089 FN:652 TP:117 FP:54
accuracy:63.075 recall:15.215
测试集
TN:4517 FN:1622 TP:495 FP:477
accuracy:70.482 recall:23.382
Epoch [2/300], train_loss: 0.6175
训练集
TN:1199 FN:500 TP:626 FP:541
accuracy:63.678 recall:55.595
验证集
TN:803 FN:358 TP:411 FP:340
accuracy:63.494 recall:53.446
测试集
TN:3111 FN:755 TP:1362 FP:1883
accuracy:62.903 recall:64.336
Epoch [3/300], train_loss: 0.5822
训练集
TN:1715 FN:783 TP:343 FP:25
accuracy:71.807 recall:30.462
验证集
TN:1131 FN:574 TP:195 FP:12
accuracy:69.351 recall:25.358
测试集
TN:4872 FN:1430 TP:687 FP:122
accuracy:78.175 recall:32.452
Epoch [4/300], train_loss: 0.5470
训练集
TN:1727 FN:775 TP:351 FP:13
accuracy:72.505 recall:31.172
验证集
TN:1131 FN:571 TP:198 FP:12
accuracy:69.508 recall:25.748
测试集
T

TN:983 FN:34 TP:735 FP:160
accuracy:89.854 recall:95.579
测试集
TN:4290 FN:82 TP:2035 FP:704
accuracy:88.947 recall:96.127
Epoch [38/300], train_loss: 0.1814
训练集
TN:1721 FN:83 TP:1043 FP:19
accuracy:96.441 recall:92.629
验证集
TN:1133 FN:54 TP:715 FP:10
accuracy:96.653 recall:92.978
测试集
TN:4802 FN:134 TP:1983 FP:192
accuracy:95.416 recall:93.67
Epoch [39/300], train_loss: 0.1389
训练集
TN:1646 FN:65 TP:1061 FP:94
accuracy:94.452 recall:94.227
验证集
TN:1087 FN:48 TP:721 FP:56
accuracy:94.561 recall:93.758
测试集
TN:4603 FN:102 TP:2015 FP:391
accuracy:93.067 recall:95.182
Epoch [40/300], train_loss: 0.1507
训练集
TN:1717 FN:110 TP:1016 FP:23
accuracy:95.359 recall:90.231
验证集
TN:1129 FN:80 TP:689 FP:14
accuracy:95.084 recall:89.597
测试集
TN:4803 FN:166 TP:1951 FP:191
accuracy:94.98 recall:92.159
Epoch [41/300], train_loss: 0.1598
训练集
TN:1722 FN:115 TP:1011 FP:18
accuracy:95.359 recall:89.787
验证集
TN:1133 FN:84 TP:685 FP:10
accuracy:95.084 recall:89.077
测试集
TN:4829 FN:172 TP:1945 FP:165
accuracy:95.261 recall

TN:4790 FN:81 TP:2036 FP:204
accuracy:95.992 recall:96.174
Epoch [75/300], train_loss: 0.0883
训练集
TN:1720 FN:52 TP:1074 FP:20
accuracy:97.488 recall:95.382
验证集
TN:1132 FN:40 TP:729 FP:11
accuracy:97.333 recall:94.798
测试集
TN:4841 FN:69 TP:2048 FP:153
accuracy:96.878 recall:96.741
Epoch [76/300], train_loss: 0.1071
训练集
TN:1718 FN:99 TP:1027 FP:22
accuracy:95.778 recall:91.208
验证集
TN:1128 FN:81 TP:688 FP:15
accuracy:94.979 recall:89.467
测试集
TN:4822 FN:152 TP:1965 FP:172
accuracy:95.444 recall:92.82
Epoch [77/300], train_loss: 0.0979
训练集
TN:1720 FN:47 TP:1079 FP:20
accuracy:97.662 recall:95.826
验证集
TN:1130 FN:33 TP:736 FP:13
accuracy:97.594 recall:95.709
测试集
TN:4848 FN:51 TP:2066 FP:146
accuracy:97.23 recall:97.591
Epoch [78/300], train_loss: 0.1237
训练集
TN:1720 FN:56 TP:1070 FP:20
accuracy:97.348 recall:95.027
验证集
TN:1132 FN:46 TP:723 FP:11
accuracy:97.019 recall:94.018
测试集
TN:4823 FN:83 TP:2034 FP:171
accuracy:96.428 recall:96.079
Epoch [79/300], train_loss: 0.0857
训练集
TN:1721 FN:52 TP:10

Epoch [112/300], train_loss: 0.1304
训练集
TN:1716 FN:46 TP:1080 FP:24
accuracy:97.558 recall:95.915
验证集
TN:1127 FN:30 TP:739 FP:16
accuracy:97.594 recall:96.099
测试集
TN:4842 FN:49 TP:2068 FP:152
accuracy:97.173 recall:97.685
Epoch [113/300], train_loss: 0.0879
训练集
TN:1717 FN:47 TP:1079 FP:23
accuracy:97.558 recall:95.826
验证集
TN:1129 FN:32 TP:737 FP:14
accuracy:97.594 recall:95.839
测试集
TN:4838 FN:47 TP:2070 FP:156
accuracy:97.145 recall:97.78
Epoch [114/300], train_loss: 0.0732
训练集
TN:1718 FN:43 TP:1083 FP:22
accuracy:97.732 recall:96.181
验证集
TN:1129 FN:32 TP:737 FP:14
accuracy:97.594 recall:95.839
测试集
TN:4844 FN:38 TP:2079 FP:150
accuracy:97.356 recall:98.205
Epoch [115/300], train_loss: 0.0878
训练集
TN:1688 FN:44 TP:1082 FP:52
accuracy:96.65 recall:96.092
验证集
TN:1120 FN:32 TP:737 FP:23
accuracy:97.123 recall:95.839
测试集
TN:4788 FN:41 TP:2076 FP:206
accuracy:96.527 recall:98.063
Epoch [116/300], train_loss: 0.0901
训练集
TN:1721 FN:47 TP:1079 FP:19
accuracy:97.697 recall:95.826
验证集
TN:1130 FN:3

Epoch [149/300], train_loss: 0.1887
训练集
TN:1725 FN:151 TP:975 FP:15
accuracy:94.208 recall:86.59
验证集
TN:1134 FN:107 TP:662 FP:9
accuracy:93.933 recall:86.086
测试集
TN:4884 FN:230 TP:1887 FP:110
accuracy:95.219 recall:89.136
Epoch [150/300], train_loss: 0.1654
训练集
TN:1718 FN:119 TP:1007 FP:22
accuracy:95.08 recall:89.432
验证集
TN:1131 FN:82 TP:687 FP:12
accuracy:95.084 recall:89.337
测试集
TN:4829 FN:163 TP:1954 FP:165
accuracy:95.387 recall:92.3
Epoch [151/300], train_loss: 0.1415
训练集
TN:1711 FN:51 TP:1075 FP:29
accuracy:97.209 recall:95.471
验证集
TN:1129 FN:32 TP:737 FP:14
accuracy:97.594 recall:95.839
测试集
TN:4800 FN:57 TP:2060 FP:194
accuracy:96.47 recall:97.308
Epoch [152/300], train_loss: 0.0923
训练集
TN:1722 FN:60 TP:1066 FP:18
accuracy:97.278 recall:94.671
验证集
TN:1133 FN:49 TP:720 FP:10
accuracy:96.914 recall:93.628
测试集
TN:4866 FN:97 TP:2020 FP:128
accuracy:96.836 recall:95.418
Epoch [153/300], train_loss: 0.1128
训练集
TN:1719 FN:61 TP:1065 FP:21
accuracy:97.139 recall:94.583
验证集
TN:1133 FN:5

Epoch [186/300], train_loss: 0.1429
训练集
TN:1716 FN:111 TP:1015 FP:24
accuracy:95.29 recall:90.142
验证集
TN:1127 FN:82 TP:687 FP:16
accuracy:94.874 recall:89.337
测试集
TN:4849 FN:192 TP:1925 FP:145
accuracy:95.261 recall:90.931
Epoch [187/300], train_loss: 0.1424
训练集
TN:1710 FN:97 TP:1029 FP:30
accuracy:95.569 recall:91.385
验证集
TN:1123 FN:71 TP:698 FP:20
accuracy:95.241 recall:90.767
测试集
TN:4812 FN:166 TP:1951 FP:182
accuracy:95.106 recall:92.159
Epoch [188/300], train_loss: 0.1260
训练集
TN:1708 FN:81 TP:1045 FP:32
accuracy:96.057 recall:92.806
验证集
TN:1118 FN:53 TP:716 FP:25
accuracy:95.921 recall:93.108
测试集
TN:4797 FN:115 TP:2002 FP:197
accuracy:95.612 recall:94.568
Epoch [189/300], train_loss: 0.1082
训练集
TN:1721 FN:66 TP:1060 FP:19
accuracy:97.034 recall:94.139
验证集
TN:1133 FN:49 TP:720 FP:10
accuracy:96.914 recall:93.628
测试集
TN:4867 FN:101 TP:2016 FP:127
accuracy:96.794 recall:95.229
Epoch [190/300], train_loss: 0.1121
训练集
TN:1720 FN:71 TP:1055 FP:20
accuracy:96.825 recall:93.694
验证集
TN:113

KeyboardInterrupt: 

In [18]:
predict_result(model,train_dataloader)

TN:1731 FN:16 TP:1105 FP:14
accuracy:98.953 recall:98.749


(98.95324494068387, 98.749)

In [14]:
predict_result(model,test_dataloader)

TN:1128 FN:12 TP:760 FP:12
accuracy:0.987 recall:0.984


(0.9874476987447699, 0.984)

In [19]:
def generate_test_data(file_dir,normal_file,abnormal_file,bert_cache_path):
    eventId_to_bert = torch.load(bert_cache_path)
    padding = torch.zeros_like(eventId_to_bert[5][1][0])
    eventId_to_bert[0] = [[], [padding]]
    sessions = []
    labels = []
    max_len = 50
    normal_data = set()
    abnormal_data = set()
    data = pd.read_csv(file_dir+normal_file, engine='c', na_filter=False, memory_map=True)
    seqs = data['EventSequence'].apply(literal_eval).tolist()
    for line in tqdm(seqs, "loading data"):
        if len(line) > 50:
            continue
        normal_data.add(tuple(line))
    data = pd.read_csv(file_dir+abnormal_file, engine='c', na_filter=False, memory_map=True)
    # blockId_list = data['BlockId'].tolist()
    seqs = data['EventSequence'].apply(literal_eval).tolist()
    for line in tqdm(seqs, "loading data"):
        if len(line) > 50:
            continue
        abnormal_data.add(tuple(line))
    for line in tqdm(normal_data, "normal:"):
        line = list(line) + [0] * (max_len - len(line))
        bert_input = []
        for id in line:
            bert_input.append(eventId_to_bert[id][1][0].cpu().numpy())
        sessions.append(tuple(bert_input))
        labels.append(0)
    for line in tqdm(abnormal_data, "abnormal:"):
        line = list(line) + [0] * (max_len - len(line))
        bert_input = []
        for id in line:
            bert_input.append(eventId_to_bert[id][1][0].cpu().numpy())
        sessions.append(tuple(bert_input))
        labels.append(1)

    print('Number of sessions({}): {}'.format(file_dir, len(sessions)))
    print('Number of normal sessions: {}'.format(len(normal_data)))
    print('Number of abnormal sessions: {}'.format(len(abnormal_data)))
    test_data = TensorDataset(torch.tensor(sessions, dtype=torch.float), torch.tensor(labels))
    # test_data = DataLoader(test_data, batch_size=batch_size)
    return test_data

In [20]:
supervised_input_normal = 'supervised_input_normal.csv'
supervised_input_abnormal = 'supervised_input_abnormal.csv'

In [21]:
test_dataset =generate_test_data(lstm_dataset,supervised_input_normal,supervised_input_abnormal,"./data/lstm/bert_official_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 2117/2117 [01:04<00:00, 32.63it/s]


Number of sessions(.\data\lstm\dataset_official\): 7111
Number of normal sessions: 4994
Number of abnormal sessions: 2117


In [22]:
test_dataloader = DataLoader(test_dataset, batch_size=2000, shuffle=True)
# test_dataloader2 = DataLoader(test_dataset2, batch_size=batch_size)

In [24]:
predict_result(model,test_dataloader)

TN:4811 FN:40 TP:2077 FP:183
accuracy:96.864 recall:91.903


(96.86401350021094, 91.903)

In [25]:
predict_result(model,test_dataloader2)

TN:1954 FN:7 TP:801 FP:83
accuracy:0.968 recall:0.906


(0.968365553602812, 0.906)

In [2]:
from unsupervised.wf_constructor import workflow_constructor,draw_wf


In [55]:
from tensorboard.backend.event_processing import event_accumulator
from torch.utils.tensorboard import SummaryWriter
ea=event_accumulator.EventAccumulator('E:\
/python_workspace//jupyter_notebook//毕业设计//dev//end_to_end_dev_v0//log/supervised_lstm_dev_v1')
ea.Reload()


In [62]:
ea.scalars.Keys()

['训练集正确率',
 '训练集召回率',
 '训练集精度',
 '训练集F1值',
 '验证集正确率',
 '验证集召回率',
 '验证集精度',
 '验证集F1值',
 '测试集集正确率',
 '测试集集召回率',
 '测试集集精度',
 '测试集集F1值',
 '第二阶段模型损失']

In [ ]:
train_acc = ea.scalars.Items('测试集准确率')
train_R = ea.scalars.Items('测试集召回率')
test_acc = ea.scalars.Items('训练集准确率')
test_R = ea.scalars.Items('训练集召回率')
Loss = ea.scalars.Items('第二阶段模型损失') 
process = {'train_acc':[[i.step,i.value] for i in train_acc]}
process['train_R'] = [[i.step,i.value] for i in train_R]
process['test_acc'] = [[i.step,i.value] for i in test_acc]
process['test_R'] = [[i.step,i.value] for i in test_R]
process['第二阶段模型训练损失'] = [[i.step,i.value] for i in Loss]

In [66]:
process = {}
for key in ea.scalars.Keys():
    process[key] =[[i.step,i.value] for i in ea.scalars.Items(key)]

In [73]:
keys =  ['测试集集正确率',
 '测试集集召回率',
 '测试集集精度',
 '测试集集F1值']
writer = SummaryWriter(log_dir='log/process/supervised/train_v3/' + '测试集')
for key in keys:
    for item in process[key][:115]:
        writer.add_scalar(key[:3]+key[4:],item[1],item[0])

In [71]:

# writer = SummaryWriter(log_dir='log/process/supervised/' + 'train_v1')
cur = ''
for key in ea.scalars.Keys():
    if cur!=key[:3]:
        writer = SummaryWriter(log_dir='log/process/supervised/train_v3/' + key[:3])
        cur = key[:3]
    for item in process[key][:115]:
        writer.add_scalar(key,item[1],item[0])


In [25]:
writer = SummaryWriter(log_dir='log/process/supervised')
for item in process['第二阶段模型训练损失'][:250]:
    writer.add_scalar('第二阶段模型训练损失',item[1],item[0])

In [16]:
import pickle
f = open('supervised_process', 'wb')
pickle.dump(process, f)

with open('supervised_process', 'rb') as supervised_process:
    process = pickle.load(supervised_process)

In [26]:
model_dir = 'model'
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_sl_name + '.pt')
print('Finished Training')

Finished Training


In [3]:
wf = workflow_constructor(model,input_size=1)
wf.generate_seq([0],1,3)


NameError: name 'model' is not defined

In [ ]:
wf.workflow_construction([0],{30})
draw_wf(wf)